<a href="https://colab.research.google.com/github/chandra303/Info-AidTech/blob/main/movie_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INFO AIDTECH

# TASK 2:  Movie Recommendation System

In [22]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('movies.csv')

In [30]:
df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [10]:
df = pd.read_csv('movies.csv',usecols=['movieId','title'])

In [31]:
df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [14]:
ratings = pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'] )

In [15]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [32]:
ratings.pivot(index='movieId',columns='userId',values='rating')

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
movies_users = ratings.pivot(index='movieId',columns='userId',values='rating').fillna(0)
movies_users.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from scipy.sparse import csr_matrix

In [20]:
mat_movies = csr_matrix(movies_users.values)
mat_movies

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)
model.fit(mat_movies)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [23]:
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [34]:
def recommender(movie_name,data,n):
  idx= process.extractOne(movie_name,df['title'])[2]
  print('Movies selected : ',df['title'][idx],'Index: ',idx)
  print('please wait....')
  distance, indices = model.kneighbors(data[idx],n_neighbors=n)
  for i in indices:
    print(df['title'][i].where(i!=idx))
    print(distance,indices)

In [35]:
recommender('toy story',mat_movies, 10)


Movies selected :  Toy Story (1995) Index:  0
please wait....
0                                                     NaN
2353                                     Indochine (1992)
418                           Hudsucker Proxy, The (1994)
615                                     Last Dance (1996)
224                             Jefferson in Paris (1995)
314                                           Cobb (1994)
322     Wonderful, Horrible Life of Leni Riefenstahl, ...
910                           Perfect Candidate, A (1996)
546     Flower of My Secret, The (La flor de mi secret...
963     Good, the Bad and the Ugly, The (Buono, il bru...
Name: title, dtype: object
[[0.         0.42739874 0.4343632  0.43573831 0.44261183 0.45290409
  0.45885465 0.4589107  0.46108723 0.46583124]] [[   0 2353  418  615  224  314  322  910  546  963]]


In [36]:
mve=input("enter the movie name to show similar 10 movies : ")
recommender(mve,mat_movies, 10)

enter the movie name to show similar 10 movies : IRON MAN
Movies selected :  Iron Man (2008) Index:  7714
please wait....
8545                            Cosmos (1980)
5585            Drums Along the Mohawk (1939)
9321           Cabin in the Woods, The (2012)
9616                      Maniac Cop 2 (1990)
8990                 Midnight in Paris (2011)
7108                          Accepted (2006)
7057       Leonard Cohen: I'm Your Man (2005)
9117    Cold Fish (Tsumetai nettaigyo) (2010)
8151                              Home (2009)
4940              Under the Tuscan Sun (2003)
Name: title, dtype: object
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] [[8545 5585 9321 9616 8990 7108 7057 9117 8151 4940]]


In [ ]:
THE ABOVE MOVIES ARE SIMILAR TO IRON MAN movies_users

THANK YOU